In [1]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [2]:
client.list_experiments()

[<Experiment: artifact_location='./mlruns/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>,
 <Experiment: artifact_location='./mlruns/2', experiment_id='2', lifecycle_stage='active', name='my-cool-experiment', tags={}>,
 <Experiment: artifact_location='mlruns/3', experiment_id='3', lifecycle_stage='active', name='random-forest-hyperopt', tags={}>]

In [3]:
client.create_experiment(name="my-cool-experiment")

MlflowException: Experiment(name=my-cool-experiment) already exists. Error: (raised as a result of Query-invoked autoflush; consider using a session.no_autoflush block if this flush is occurring prematurely)
(sqlite3.IntegrityError) UNIQUE constraint failed: experiments.name
[SQL: INSERT INTO experiments (name, artifact_location, lifecycle_stage) VALUES (?, ?, ?)]
[parameters: ('my-cool-experiment', None, 'active')]
(Background on this error at: https://sqlalche.me/e/14/gkpj)

In [4]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string='metrics.rmse < 6.8',
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [5]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 80c25552b0c6432a8e3a216950ba79aa, rmse: 6.2935
run id: ecbe8665d6dd4455b0162966ae539e87, rmse: 6.2935
run id: 174e6f474ade4e4a91092f4308b3c7ef, rmse: 6.3042
run id: 09f987f0a8a7410c84aa07dfb3301f0c, rmse: 6.3063
run id: 23500ff06ce2404cba73b5cc133ec648, rmse: 6.3100


In [6]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [7]:
run_id = '071861c5096e4607a7562ba17f91e902'
model_uri = f'runs:/{run_id}/model'

mlflow.register_model(model_uri=model_uri, name='nyc-taxi-regressor')

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
2022/05/29 16:32:02 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: nyc-taxi-regressor, version 4
Created version '4' of model 'nyc-taxi-regressor'.


<ModelVersion: creation_timestamp=1653813122936, current_stage='None', description=None, last_updated_timestamp=1653813122936, name='nyc-taxi-regressor', run_id='071861c5096e4607a7562ba17f91e902', run_link=None, source='./mlruns/1/071861c5096e4607a7562ba17f91e902/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=4>

In [8]:
model_name = 'nyc-taxi-regressor'
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: Production
version: 2, stage: Archived
version: 3, stage: Staging
version: 4, stage: None


In [9]:
client.transition_model_version_stage(
    name=model_name,
    version='3',
    stage='Staging',
    archive_existing_versions=False
)

<ModelVersion: creation_timestamp=1653449634270, current_stage='Staging', description='The model version 3 is transitioned to Staging on 2022-05-25 11:39:48.652658', last_updated_timestamp=1653813134416, name='nyc-taxi-regressor', run_id='071861c5096e4607a7562ba17f91e902', run_link=None, source='./mlruns/1/071861c5096e4607a7562ba17f91e902/artifacts/mode;', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [10]:
from datetime import datetime

date = datetime.today()

model_version = 3
new_stage = 'Staging'

client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} is transitioned to {new_stage} on {date}"
)

<ModelVersion: creation_timestamp=1653449634270, current_stage='Staging', description='The model version 3 is transitioned to Staging on 2022-05-29 16:32:17.889238', last_updated_timestamp=1653813137890, name='nyc-taxi-regressor', run_id='071861c5096e4607a7562ba17f91e902', run_link=None, source='./mlruns/1/071861c5096e4607a7562ba17f91e902/artifacts/mode;', status='READY', status_message=None, tags={}, user_id=None, version=3>

In [11]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [12]:
run_id = 'b0e958e985e741209038883970380ec5'
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')

'/home/gojef/mlops-study/02-experiment-tracking/preprocessor'

In [13]:
df = read_dataframe('data/green_tripdata_2021-03.parquet')
df.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,duration
0,2,2021-03-01 00:05:42,2021-03-01 00:14:03,N,1.0,83,129,1.0,1.56,7.5,...,0.5,0.00,0.0,NaN,0.3,8.80,1.0,1.0,0.0,8.350000
1,2,2021-03-01 00:21:03,2021-03-01 00:26:17,N,1.0,243,235,1.0,0.96,6.0,...,0.5,0.00,0.0,NaN,0.3,7.30,2.0,1.0,0.0,5.233333
2,2,2021-03-01 00:02:06,2021-03-01 00:22:26,N,1.0,75,242,1.0,9.93,28.0,...,0.5,2.00,0.0,NaN,0.3,31.30,1.0,1.0,0.0,20.333333
3,2,2021-03-01 00:24:03,2021-03-01 00:31:43,N,1.0,242,208,1.0,2.57,9.5,...,0.5,0.00,0.0,NaN,0.3,10.80,2.0,1.0,0.0,7.666667
4,1,2021-03-01 00:11:10,2021-03-01 00:14:46,N,1.0,41,151,1.0,0.80,5.0,...,0.5,1.85,0.0,NaN,0.3,8.15,1.0,1.0,0.0,3.600000


In [14]:
import pickle

with open('preprocessor/preprocessor.b', 'rb') as f_in:
    dv = pickle.load(f_in)

In [15]:
X_test = preprocess(df, dv)

In [16]:
target = 'duration'
y_test = df[target].values

In [17]:
model_name

'nyc-taxi-regressor'

In [18]:
%time test_model(name=model_name, stage='Production', X_test=X_test, y_test=y_test)

[16:32:32] WARNING: ../src/objective/regression_obj.cu:203: reg:linear is now deprecated in favor of reg:squarederror.
CPU times: user 19.1 s, sys: 43.8 ms, total: 19.2 s
Wall time: 2.86 s


{'rmse': 6.241331628812532}

In [19]:
%time test_model(name=model_name, stage='None', X_test=X_test, y_test=y_test)

CPU times: user 6.15 s, sys: 643 ms, total: 6.79 s
Wall time: 7.24 s


{'rmse': 6.88560552836832}

In [20]:
%time test_model(name=model_name, stage='Staging', X_test=X_test, y_test=y_test)

OSError: No such file or directory: './mlruns/1/071861c5096e4607a7562ba17f91e902/artifacts/mode;/.'

In [38]:
client.transition_model_version_stage(
    name=model_name,
    version=1,
    stage='Production',
    archive_existing_versions=True
)

<ModelVersion: creation_timestamp=1653448512134, current_stage='Production', description='', last_updated_timestamp=1653457628582, name='nyc-taxi-regressor', run_id='80c25552b0c6432a8e3a216950ba79aa', run_link='', source='./mlruns/1/80c25552b0c6432a8e3a216950ba79aa/artifacts/models_mlflow', status='READY', status_message=None, tags={'model': 'xgboost'}, user_id=None, version=1>